In [97]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  #显示中文
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
plt.rcParams['figure.dpi'] = 300
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # 显示所有列
from tqdm.notebook import tqdm
import geopandas as gpd
import transbigdata as tbd
import geoplot as gplt

In [25]:
# 读取四环数据，用于裁切，本文的数据是四环内的数据
sihuan = gpd.read_file('E:/Data/成都数据/成都道路_好用/成都四环/四环.shp')
sihuan = sihuan.to_crs(epsg=4326)

In [101]:
real_trip = pd.read_csv('E:/Data/mobike201803/trip_data_3_7.csv')
real_trip['startlng'] = tbd.gcj02towgs84(real_trip['startlng'], real_trip['startlat'])[0]
real_trip['startlat'] = tbd.gcj02towgs84(real_trip['startlng'], real_trip['startlat'])[1]
real_trip = gpd.GeoDataFrame(real_trip, geometry=gpd.points_from_xy(real_trip.startlng, real_trip.startlat))
real_trip_in_sihuan = real_trip.clip(mask=sihuan)

### 将trip匹配到网格上

In [106]:
grid_m_list = np.append((np.linspace(100,900,9)),\
          np.linspace(1000,5000,9)) # 生成网格大小列表

for samp_rate in tqdm(range(2,21)):
    # 读取trip数据
    trip = pd.read_csv('E:/Data/mobike201803/location_to_trip_data_new_gcj2wgs/loc2trip_%d.csv'%samp_rate,encoding='utf-8-sig')
    trip = gpd.GeoDataFrame(trip, geometry=gpd.points_from_xy(trip['start_lng'], trip['start_lat']), crs='epsg:4326')
    trip_in_sihuan = trip.clip(mask=sihuan)

    # 将trip_in_sihuan连接到grid上
    for grid_m in grid_m_list:
        grid_m = int(grid_m)
        print('采样率： %dmin  and'%samp_rate, '  网格大小：', grid_m, '米', end='----')

        # 读取网格
        grid = pd.read_csv('E:/Data/成都数据/rect_road_poi/rect%d.csv'%grid_m,encoding='utf-8-sig')
        from shapely import wkt
        grid['geometry'] = grid['geometry'].apply(wkt.loads)
        grid = gpd.GeoDataFrame(grid, geometry='geometry', crs='epsg:4326')

        # 将trip_in_sihuan连接到当前grid上，对应generate_trip_num 
        dfsjoin = gpd.sjoin(grid, trip_in_sihuan)  # Spatial join Points to polygons
        # 根据poly_id分组，计算每个网格内的POI数量
        dfpivot = pd.pivot_table(dfsjoin, index='poly_id', aggfunc={'bikeid': len})
        # 重命名列名
        dfpivot.rename(columns={'bikeid': 'generate_trip_num'}, inplace=True)
        grid_trip_num = grid.merge(dfpivot, how='left', on='poly_id') 

        # 将real_trip_in_sihuan连接到当前grid上 ，对应real_trip_num  
        dfsjoin = gpd.sjoin(grid, real_trip_in_sihuan)  # Spatial join Points to polygons
        # 根据poly_id分组，计算每个网格内的POI数量
        dfpivot = pd.pivot_table(dfsjoin, index='poly_id', aggfunc={'bikeid': len})
        # 重命名列名
        dfpivot.rename(columns={'bikeid': 'real_trip_num'}, inplace=True)
        grid_trip_num = grid_trip_num.merge(dfpivot, how='left', on='poly_id') 


        
        print('网格内的trip总数量对比： real-generate{} vs {}'.format(real_trip_in_sihuan.shape[0] , trip_in_sihuan.shape[0] ) )
        grid_trip_num = grid_trip_num.fillna(0)
        # 多个poi列求和为一列
        grid_trip_num['poi_num'] = grid_trip_num.iloc[:,4:-2].sum(axis=1)
        # 保存
        grid_trip_num.to_csv('E:/Data/mobike201803/sample_rate_grid/sample_rate{}_rect{}.csv'.format(samp_rate, grid_m) ,index=False,encoding='utf-8-sig')



  0%|          | 0/19 [00:00<?, ?it/s]

采样率： 2min  and   网格大小： 100 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 200 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 300 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 400 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 500 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 600 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 700 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 800 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 900 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 1000 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 1500 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 2000 米----网格内的trip总数量对比： real-generate700545 vs 659093
采样率： 2min  and   网格大小： 2500 米----网格内的trip总数量对比： real-generate700545 vs 65

### 添加一些列

In [5]:
grid_m_list = np.append((np.linspace(100,900,9)),\
          np.linspace(1000,5000,9)) # 生成网格大小列表

for samp_rate in tqdm(range(2,21)):
    # 读取trip数据
    for grid_m in grid_m_list:
        grid_m = int(grid_m)
        print('采样率： %dmin  and'%samp_rate, '  网格大小：', grid_m, '米')
        samp_rate_grid = pd.read_csv('E:/Data/mobike201803/sample_rate_grid/sample_rate{}_rect{}.csv'.format(samp_rate,grid_m),encoding='utf-8-sig')
        from shapely import wkt
        samp_rate_grid['geometry'] = samp_rate_grid['geometry'].apply(wkt.loads)
        samp_rate_grid = gpd.GeoDataFrame(samp_rate_grid, geometry='geometry', crs='epsg:4326')
        samp_rate_grid = samp_rate_grid.to_crs('epsg:32648')
        samp_rate_grid['area'] = samp_rate_grid['geometry'].area / 1000000 # 平方公里

        # 添加列
        samp_rate_grid['samp_rate'] = samp_rate # 采样率
        samp_rate_grid['grid_m'] = grid_m # 网格大小
        # poi密度
        samp_rate_grid['poi_density'] = samp_rate_grid['poi_num'] / samp_rate_grid['area'] # poi密度
        # trip比例
        samp_rate_grid['trip_ratio'] = samp_rate_grid['real_trip_num'] / samp_rate_grid['generate_trip_num'] # trip比例
        # 转为wgs84坐标系
        samp_rate_grid = samp_rate_grid.to_crs('epsg:4326')
        # 保存
        samp_rate_grid.to_csv('E:/Data/mobike201803/sample_rate_grid/sample_rate{}_rect{}.csv'.format(samp_rate,grid_m),index=False,encoding='utf-8-sig')


  0%|          | 0/19 [00:00<?, ?it/s]

采样率： 2min  and   网格大小： 100 米
采样率： 2min  and   网格大小： 200 米
采样率： 2min  and   网格大小： 300 米
采样率： 2min  and   网格大小： 400 米
采样率： 2min  and   网格大小： 500 米
采样率： 2min  and   网格大小： 600 米
采样率： 2min  and   网格大小： 700 米
采样率： 2min  and   网格大小： 800 米
采样率： 2min  and   网格大小： 900 米
采样率： 2min  and   网格大小： 1000 米
采样率： 2min  and   网格大小： 1500 米
采样率： 2min  and   网格大小： 2000 米
采样率： 2min  and   网格大小： 2500 米
采样率： 2min  and   网格大小： 3000 米
采样率： 2min  and   网格大小： 3500 米
采样率： 2min  and   网格大小： 4000 米
采样率： 2min  and   网格大小： 4500 米
采样率： 2min  and   网格大小： 5000 米
采样率： 3min  and   网格大小： 100 米
采样率： 3min  and   网格大小： 200 米
采样率： 3min  and   网格大小： 300 米
采样率： 3min  and   网格大小： 400 米
采样率： 3min  and   网格大小： 500 米
采样率： 3min  and   网格大小： 600 米
采样率： 3min  and   网格大小： 700 米
采样率： 3min  and   网格大小： 800 米
采样率： 3min  and   网格大小： 900 米
采样率： 3min  and   网格大小： 1000 米
采样率： 3min  and   网格大小： 1500 米
采样率： 3min  and   网格大小： 2000 米
采样率： 3min  and   网格大小： 2500 米
采样率： 3min  and   网格大小： 3000 米
采样率： 3min  and   网格大小： 3500 米
采样率： 3min  and   网格大小： 4000 